In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch14
!touch ch14/__init__.py

mkdir: cannot create directory ‘ch14’: File exists


In [0]:
import imp

## 14.0 序论

- 迭代是数据处理的基石
- 扫描内存中放不下的数据集时，我们要找到一种惰性获取数据项的方式，即按需一次获取一个数据项
  - 这就是迭代器模式（Iterator pattern）
- Python 中迭代器与生成器的关系
  - 所有生成器都是迭代器，因为生成器完全实现了迭代器接口
  - 迭代器用于从集合中取出元素；而生成器用于“凭空”生成元素
- 在 Python 语言内部，迭代器用于支持：
  - `for` 循环
  - 构建和扩展集合类型
  - 逐行遍历文本文件
  - 列表推导、字典推导和集合推导
  - 元组拆包
  - 调用函数时，使用 `*` 拆包实参

## 14.1 `Sentence` 类第 1 版： 单词序列

###### 示例 14-1　sentence.py：把句子划分为单词序列

In [0]:
%%writefile ch14/sentence.py
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)  # 返回一个字符串列表，里面的元素是正则表达式的全部非重叠匹配
  
  def __getitem__(self, index):
    return self.words[index]

  def __len__(self):  # 为了完善序列协议，需要 __len__ 方法；但如果只是让对象可以迭代，没必要实现此方法
    return len(self.words)

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)  # 用于生成大型数据结构的简略字符串表示形式

Overwriting ch14/sentence.py


###### 示例 14-2　测试 Sentence 实例能否迭代

In [0]:
import ch14.sentence
imp.reload(ch14.sentence)
from ch14.sentence import Sentence

In [0]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [0]:
for word in s:  # Sentence 实例可以迭代
  print(word)

The
time
has
come
the
Walrus
said


In [0]:
list(s)  # 因为可以迭代，所以 Sentence 对象可以用于构建列表和其他可迭代的类型

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 序列可以迭代的原因： `iter` 函数

- 解释器需要迭代对象 `x` 时，会自动调用 `iter(x)`
- 内置的 `iter` 函数的作用如下：
  1. 检查对象是否实现了 `__iter__` 方法，如果实现了就调用它，获取一个迭代器
  2. 如果没有实现 `__iter__` 方法，但是实现了 `__getitem__` 方法，Python 会创建一个迭代器，尝试按顺序（从索引 0 开始）获取元素
  3. 如果尝试失败，Python 抛出 `TypeError` 异常，通常会提示“C object is not iterable”（C 对象不可迭代），其中 C 是目标对象所属的类
- 任何 Python 序列都可迭代的原因是，它们都实现了 `__getitem__` 方法
  - 标准的序列也都实现了 `__iter__` 方法，因此自定义的序列也应该实现此方法
- 在白鹅类型（goose-typing）理论中， 如果实现了 `__iter__` 方法，那么就认为对象是可迭代的
  - 不需要创建子类，也不用注册，因为 `abc.Iterable` 类实现了 `__subclasshook__` 方法

In [0]:
class Foo:
  def __iter__(self):
    pass


from collections import abc
issubclass(Foo, abc.Iterable)

True

In [0]:
f = Foo()
isinstance(f, abc.Iterable)

True

- 　从 Python 3.4 开始，检查对象 `x` 能否迭代，最准确的方法是：调用 `iter(x)` 函数
  - 如果不可迭代，再处理 `TypeError` 异常
  - 这比使用 `isinstance(x, abc.Iterable)`
    - 因为 `iter(x)` 函数会考虑到遗留的 `__getitem__` 方法
    - 而 `abc.Iterable` 类则不考虑

## 14.2 可迭代对象与迭代器的对比

- 可迭代对象
  - 使用 `iter` 内置函数可以获取迭代器的对象
    - 如果对象实现了能返回迭代器的 `__iter__` 方法，那么对象就是可迭代的
    - 序列都可以迭代
    - 实现了 `__getitem__` 方法，而且其参数是从零开始的索引，这种对象也可以迭代
- 迭代器
  - 迭代器是这样的对象：实现了无参数的 `__next__` 方法，返回序列中的下一个元素；
  - 如果没有元素了，那么抛出 `StopIteration` 异常。
  - Python 中的迭代器还实现了 `__iter__` 方法，因此迭代器也可以迭代

- 标准的迭代器接口有两个方法
  - `__next__`
    - 返回下一个可用的元素，如果没有元素，则抛出 `StopIteration`　异常
    - Python 语言内部会处理 `for` 循环和其他迭代上下文（如列表推导、元组拆包，等等）中的 `StopIteration` 异常
  - `__iter__`
    - 返回 `self`， 以便在应该使用可迭代对象的地方使用迭代器，例如，在 `for` 循环中
- 接口在 `collections.abc.Iterator` 抽象基类中制定，如下图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200525111946.png width=700>

###### 示例 14-3　`abc.Iterator` 类，摘自 [Lib/_collections_abc.py](https://hg.python.org/cpython/file/3.4/Lib/_collections_abc.py#l93)

```python
  class Iterator(Iterable):

    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented
```

- 由于实现了 `Iterator.__subclasshook__` 方法，即使对象 `x` 所属的类不是 `Iterator` 类的真实子类或虚拟子类，也能通过下述语句进行检查
  - `isinstance(x, abc.Iterator)`

###### 迭代器对象的构建与使用示例

In [0]:
from ch14.sentence import Sentence

In [0]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)  # 从 s3 中获取迭代器
it

In [0]:
next(it)

'Pig'

In [0]:
next(it)

'and'

In [0]:
next(it)

'Pepper'

In [0]:
next(it)

StopIteration: ignored

In [0]:
list(it)  # 到头后，迭代器即失去作用

[]

In [0]:
list(iter(s3))  # 如果想再次迭代，需要重新构造迭代器

['Pig', 'and', 'Pepper']

## 14.3 `Sentence` 类第 2 版： 典型的迭代器

###### 示例 14-4　sentence_iter.py：使用迭代器模式实现 `Sentence` 类

In [0]:
%%writefile ch14/sentence_iter.py
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    return SentenceIterator(self.words)  # 根据可迭代化协议， __iter__ 方法实例化并返回一个迭代器


class SentenceIterator:
  def __init__(self, words):
    self.words = words
    self.index = 0

  def __next__(self):
    try:
      word = self.words[self.index]
    except IndexError:
      raise StopIteration()
    self.index += 1
    return word

  def __iter__(self):
    return self

Overwriting ch14/sentence_iter.py


In [0]:
import ch14.sentence_iter
imp.reload(ch14.sentence_iter)
from ch14.sentence_iter import Sentence

In [0]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [0]:
for word in s:  # Sentence 实例可以迭代
  print(word)

The
time
has
come
the
Walrus
said


In [0]:
list(s)  # 因为可以迭代，所以 Sentence 对象可以用于构建列表和其他可迭代的类型

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 把 `Sentence` 变成迭代器： 坏主意

- 迭代器可以迭代，但是可迭代对象一定不能是自身的迭代器
  - 即可迭代的对象必须实现 `__iter__` 方法，但不能实现 `__next__` 方法
  - 迭代器应该一直可以迭代。迭代器的 `__iter__` 方法应该返回自身

- 迭代器模式可用来：
  - 访问一个聚合对象的内容而无需暴露它的内部表示
  - 支持对聚合对象的多种遍历
  - 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）
- 为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，
  - 而且各个迭代器要能维护自身的内部状态
  - 因此这一模式正确的实现方式是，每次调用 `iter(my_iterable)` 都新建一个独立的迭代器

## 14.4 `Sentence` 类的第 3 版： 生成器函数

###### 示例 14-5　sentence_gen.py：使用生成器函数实现 `Sentence` 类

- 此例中 `__iter__` 方法是生成器函数，调用时会构建一个实现了迭代器接口的生成器对象，因此不需再定义 `SentenceIterator` 类

In [0]:
%%writefile ch14/sentence_gen.py
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    for word in self.words:
      yield word
    return # 不是必须的，不管有没有 return 语句，生成器函数都不会抛出 StopIteration 异常，而是在生成完全部值后直接退出
    

Overwriting ch14/sentence_gen.py


In [0]:
import ch14.sentence_gen
imp.reload(ch14.sentence_gen)
from ch14.sentence_gen import Sentence

In [0]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [0]:
for word in s:  # Sentence 实例可以迭代
  print(word)

The
time
has
come
the
Walrus
said


In [0]:
list(s)  # 因为可以迭代，所以 Sentence 对象可以用于构建列表和其他可迭代的类型

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 生成器函数的工作原理

- 只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函数
  - 调用生成器函数时，会返回一个生成器对象
  - 即生成器函数是生成器工厂
- 生成器函数的行为
  - 调用生成器函数会返回生成器
  - 生成器产出或生成值
  - 生成器函数定义体中的 `return` 语句会触发生成器对象抛出 `StopIteration` 异常

###### 生成器的行为

In [0]:
def gen_123():
  yield 1  # 生成器函数的定义体中通常都有循环，不过这不是必要条件
  yield 2
  yield 3

In [0]:
gen_123  # gen_123 是函数对象

<function __main__.gen_123>

In [0]:
gen_123()  # 返回的是一个生成器对象

<generator object gen_123 at 0x7f6f8c309f68>

In [0]:
for i in gen_123():  # 生成器是迭代器，会生成传给 yield 关键字的表达式的值
  print(i)

1
2
3


In [0]:
g = gen_123()

In [0]:
next(g)

1

In [0]:
next(g)

2

In [0]:
next(g)

3

In [0]:
next(g)

StopIteration: ignored

###### 示例 14-6　运行时打印消息的生成器函数

In [0]:
def gen_AB():
  print('start')
  yield 'A'
  print('continue')
  yield 'B'
  print('end.')

In [0]:
for c in gen_AB():
  print('-->', c)

start
--> A
continue
--> B
end.


## 14.5 Sentence 类的第 4 版： 惰性实现

- 设计 `Iterator` 接口时考虑到了惰性：`next(my_iterator)` 一次生成一个元素
- 惰性求值（lazy evaluation）和及早求值（eager evaluation）是编程语言理论方面的技术术语

###### 示例 14-7　sentence_gen2.py： 在生成器函数中调用 `re.finditer` 生成器函数，实现 `Sentence` 类

- `re.finditer` 函数是 `re.findall` 函数的惰性版本，返回的不是列表，而是一个生成器，按需生成 `re.MatchObject` 实例
- 如果有很多匹配，`re.finditer` 函数能节省大量内存。

In [0]:
%%writefile ch14/sentence_gen2.py
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
  def __init__(self, text):
    self.text = text

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    for match in RE_WORD.finditer(self.text):
      yield match.group()  # match.group() 方法从 MatchObject 实例中提取匹配正则表达式的具体文本

Overwriting ch14/sentence_gen2.py


In [0]:
import ch14.sentence_gen2
imp.reload(ch14.sentence_gen2)
from ch14.sentence_gen2 import Sentence

In [0]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [0]:
for word in s:  # Sentence 实例可以迭代
  print(word)

The
time
has
come
the
Walrus
said


In [0]:
list(s)  # 因为可以迭代，所以 Sentence 对象可以用于构建列表和其他可迭代的类型

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.6　`Sentence` 类第5版：生成器表达式

- 生成器表达式可以理解为列表推导的惰性版本：
  - 其不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素
- 生成器表达式会产出生成器
- 生成器表达式是语法糖：其完全可以替换为生成器函数，不过有时使用生成器表达式更便利

###### 示例 14-8　先在列表推导中使用 `gen_AB` 生成器函数，然后在生成器表达式中使用

In [0]:
def gen_AB():
  print('start')
  yield 'A'
  print('continue')
  yield 'B'
  print('end.')

In [0]:
res1 = [x*3 for x in gen_AB()]  # 列表推导迫切地迭代 gen_AB() 函数生成的生成器对象产出的元素：'A' 和 'B'。

start
continue
end.


In [0]:
for res in res1:
  print("-->", res)

--> AAA
--> BBB


In [0]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x7f6f8c323e60>

In [0]:
for res in res2:  # 只有 for 循环迭代 res2 时，gen_AB 函数的定义体才会真正执行
  print("-->", res)

start
--> AAA
continue
--> BBB
end.


###### 示例 14-9　sentence_genexp.py：使用生成器表达式实现 `Sentence` 类

In [0]:
%%writefile ch14/sentence_genexp.py
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
  def __init__(self, text):
    self.text = text

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    return (match.group() for match in RE_WORD.finditer(self.text))

Writing ch14/sentence_genexp.py


In [0]:
import ch14.sentence_genexp
imp.reload(ch14.sentence_genexp)
from ch14.sentence_genexp import Sentence

In [0]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [0]:
for word in s:  # Sentence 实例可以迭代
  print(word)

The
time
has
come
the
Walrus
said


In [0]:
list(s)  # 因为可以迭代，所以 Sentence 对象可以用于构建列表和其他可迭代的类型

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.7 何时使用生成器表达式

- 生成器表达式是创建生成器的简洁句法，这样无需先定义函数再调用
- 如果生成器表达式要分成多行写，则最好定义生成器函数，以便提高可读性
- 生成器函数有名称，因此可以重用

- 如果函数或构造方法只有一个参数，传入生成器表达式时不用写一对调用函数的括号，再写一对括号围住生成器表达式，只写一对括号就行了
- 如果生成器表达式后面还有其他参数，那么必须使用括号围住，否则会抛出 SyntaxError 异常

## 14.8 另一个示例： 等差数列生成器

- 典型的迭代器模式作用很简单——遍历数据结构
- 获取序列中即时生成的下一个值时，也可用到迭代器这种基于方法的标准接口

###### 示例 14-11　`ArithmeticProgression` 类

In [0]:
%%writefile ch14/arithmetic_progression.py
class ArithmeticProgression:
  
  def __init__(self, begin, step, end=None):
    self.begin = begin
    self.step = step
    self.end = end  # None -> 无穷数列

  def __iter__(self):
    result = type(self.begin + self.step)(self.begin)  # 为了让数列的首项与其他项的类型一样，先做加法运算，然后使用计算结果的类型强制转换生成的结果
    forever = self.end is None
    index = 0
    while forever or result < self.end:
      yield result
      index += 1
      # 没有直接使用 self.step 不断地增加 result，而是选择使用 index 变量，
      # 把 self.begin 与 self.step 和 index 的乘积相加，计算 result 的各个值，
      # 以此降低处理浮点数时累积效应致错的风险。
      result = self.begin + self.step * index  

Writing ch14/arithmetic_progression.py


In [0]:
import ch14.arithmetic_progression
imp.reload(ch14.arithmetic_progression)
from ch14.arithmetic_progression import ArithmeticProgression

###### 示例 14-10　演示 `ArithmeticProgression` 类的用法

In [0]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [0]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [0]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [0]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [0]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

###### 示例 14-12　`aritprog_gen` 生成器函数

In [0]:
%%writefile ch14/aritprog_gen.py
def aritprog_gen(begin, step, end=None):
  result = type(begin + step)(begin)
  forever = end is None
  index = 0
  while forever or result < end:
    yield result
    index += 1
    result = begin + step * index

Overwriting ch14/aritprog_gen.py


- 测试

In [0]:
import ch14.aritprog_gen
imp.reload(ch14.aritprog_gen)
from ch14.aritprog_gen import aritprog_gen

In [0]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

In [0]:
from fractions import Fraction
ap = aritprog_gen(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

### 使用 `itertools` 模块生成等差数列

- Python 3.4 中的 `itertools` 模块提供了 $19$ 个生成器函数，结合起来使用能实现很多有用的用法

##### `itertools.count` 函数返回的生成器能生成多个数

- `itertools.count` 函数会生成从零开始的整数数列
- `itertools.count` 函数从不停止

In [0]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [0]:
next(gen)

1.5

In [0]:
next(gen)

2.0

In [0]:
next(gen)

2.5

##### `itertools.takewhile` 函数会生成一个使用另一个生成器的生成器，在指定的条件计算结果为 False 时停止

In [0]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

###### 示例 14-13　aritprog_v3.py：与前面的 `aritprog_gen` 函数作用相同

In [0]:
%%writefile ch14/aritprog_v3.py
import itertools

def aritprog_gen(begin, step, end=None):
  first = type(begin + step)(begin)
  ap_gen = itertools.count(first, step)
  if end is not None:
    ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
  return ap_gen

Overwriting ch14/aritprog_v3.py


In [0]:
import ch14.aritprog_v3
imp.reload(ch14.aritprog_v3)
from ch14.aritprog_v3 import aritprog_gen

In [0]:
ap = aritprog_gen(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [0]:
from fractions import Fraction
ap = aritprog_gen(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

## 14.9 标准库中的生成器函数

### 用于过滤的生成器函数

###### 表14-1：用于过滤的生成器函数

| 模块        | 函数                                                    | 说明                                                                                        |
|-----------|-------------------------------------------------------|-------------------------------------------------------------------------------------------|
| itertools | compress\(it, selector\_it\)                          | 并行处理两个可迭代的对象；如果 selector\_it中的元素是真值，产出 it 中对应的元素                                          |
| itertools | dropwhile\(predicate, it\)                            | 处理 it，跳过 predicate 的计算结果为真值的元素，然后产出剩下的各个元素（不再进一步检查）, 相当于 drop while False                                     |
| （内置）      | filter\(predicate, it\)                               | 把 it 中的各个元素传给 predicate，如果 predicate\(item\) 返回真值，那么产出对应的元素；如果 predicate 是 None，那么只产出真值元素 |
| itertools | filterfalse\(predicate, it\)                          | 与 filter 函数的作用类似，不过 predicate 的逻辑是相反的：predicate 返回假值时产出对应的元素                              |
| itertools | islice\(it, stop\) 或 islice\(it, start,stop, step=1\) | 产出 it 的切片，作用类似于 s\[:stop\] 或 s\[start:stop:step\]，不过 it 可以是任何可迭代的对象，而且这个函数实现的是惰性操作        |
| itertools | takewhile\(predicate,it\)                             | predicate 返回真值时产出对应的元素，然后立即停止，不再继续检查，相当于 take while False                                                     |


###### 示例 14-14　演示用于过滤的生成器函数

In [0]:
def vowel(c):
  return c.lower() in 'aeiou'

In [0]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [0]:
import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [0]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [0]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [0]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [0]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [0]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [0]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

### 用于映射的生成器函数

###### 表14-2：用于映射的生成器函数

| 模块        | 函数                                    | 说明                                                                           |
|-----------|---------------------------------------|------------------------------------------------------------------------------|
| itertools | accumulate\(it,\[func\]\)             | 产出累积的总和；如果提供了 func，那么把前两个元素传给它，然后把计算结果和下一个元素传给它，以此类推，最后产出结果                  |
| （内置）      | enumerate\(iterable,start=0\)         | 产出由两个元素组成的元组，结构是 \(index, item\)，其中 index 从 start 开始计数，item 则从 iterable 中获取  |
| （内置）      | map\(func, it1,\[it2, \.\.\., itN\]\) | 把 it 中的各个元素传给func，产出结果；如果传入 N 个可迭代的对象，那么 func 必须能接受 N 个参数，而且要并行处理各个可迭代的对象    |
| itertools | starmap\(func, it\)                   | 把 it 中的各个元素传给 func，产出结果；输入的可迭代对象应该产出可迭代的元素 iit，然后以 func\(\*iit\) 这种形式调用 func |


###### 示例 14-15　演示 itertools.accumulate 生成器函数

In [0]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [0]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [0]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [0]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [0]:
list(itertools.accumulate(range(1, 11), operator.mul))  # 计算各个数的阶乘

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

###### 示例 14-16　演示用于映射的生成器函数

In [0]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [0]:
import operator
list(map(operator.mul, range(11), [2, 4, 8]))  # 元素最少的那个可迭代对象到头后就停止

[0, 4, 16]

In [0]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [0]:
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [0]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))  # 计算平均值

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### 合并多个可迭代对象的生成器函数

###### 表14-3：合并多个可迭代对象的生成器函数

| 模块        | 函数                                              | 说明                                                                               |
|-----------|-------------------------------------------------|----------------------------------------------------------------------------------|
| itertools | chain\(it1, \.\.\., itN\)                       | 先产出 it1 中的所有元素，然后产出 it2 中的所有元素，以此类推，无缝连接在一起                                      |
| itertools | chain\.from\_iterable\(it\)                     | 产出 it 生成的各个可迭代对象中的元素，一个接一个，无缝连接在一起；it 应该产出可迭代的元素，例如可迭代的对象列表                      |
| itertools | product\(it1, \.\.\., itN,repeat=1\)            | 计算笛卡儿积：从输入的各个可迭代对象中获取元素，合并成由 N 个元素组成的元组，与嵌套的 for 循环效果一样；repeat 指明重复处理多少次输入的可迭代对象 |
| （内置）      | zip\(it1, \.\.\., itN\)                         | 并行从输入的各个可迭代对象中获取元素，产出由 N 个元素组成的元组，只要有一个可迭代的对象到头了，就默默地停止                          |
| itertools | zip\_longest\(it1, \.\.\.,itN, fillvalue=None\) | 并行从输入的各个可迭代对象中获取元素，产出由 N 个元素组成的元组，等到最长的可迭代对象到头后才停止，空缺的值使用 fillvalue填充            |


###### 示例 14-17　演示用于合并的生成器函数

In [0]:
import itertools
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [0]:
list(itertools.chain(enumerate('ABC')))  # 如果传入的只是一个可迭代对象， chain 函数没有什么作用

[(0, 'A'), (1, 'B'), (2, 'C')]

In [0]:
list(itertools.chain.from_iterable(enumerate('ABC')))  # 从可迭代对象中获取每个元素，前提是各个元素本身也是可迭代对象

[0, 'A', 1, 'B', 2, 'C']

In [0]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [0]:
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [0]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [0]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

###### 示例 14-18　演示 itertools.product 生成器函数

In [0]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [0]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [0]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [0]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [0]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [0]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


### 把输入的各个元素扩展成多个输出元素

| 模块        | 函数                                              | 说明                                                                   |
|-----------|-------------------------------------------------|----------------------------------------------------------------------|
| itertools | combinations\(it, out\_len\)                    | 把 it 产出的 out\_len 个元素组合在一起，然后产出                                      |
| itertools | combinations\_with\_replacement\(it, out\_len\) | 把 it 产出的 out\_len 个元素组合在一起，然后产出，包含相同元素的组合                            |
| itertools | count\(start=0, step=1\)                        | 从 start 开始不断产出数字，按 step 指定的步幅增加                                      |
| itertools | cycle\(it\)                                     | 从 it 中产出各个元素，存储各个元素的副本，然后按顺序重复不断地产出各个元素                              |
| itertools | permutations\(it, out\_len=None\)               | 把 out\_len 个 it 产出的元素排列在一起，然后产出这些排列；out\_len的默认值等于 len\(list\(it\)\) |
| itertools | repeat\(item, \[times\]\)                       | 重复不断地产出指定的元素，除非提供 times，指定次数                                         |


###### 示例 14-19　演示 count、repeat 和 cycle 的用法

In [0]:
import itertools
ct = itertools.count()
next(ct)

0

In [0]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [0]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [0]:
cy = itertools.cycle('ABC')
next(cy)

'A'

In [0]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [0]:
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [0]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [0]:
import operator
list(map(operator.mul, range(11), itertools.repeat(5)))  # 将每个数都乘以 5

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

###### 示例 14-20　组合学生成器函数会从输入的各个元素中产出多个值

In [0]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [0]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [0]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [0]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### 用于重新排列元素的生成器函数

###### 表 14-5 用于重新排列元素的生成器函数

| 模块        | 函数                     | 说明                                                                |
|-----------|------------------------|-------------------------------------------------------------------|
| itertools | groupby\(it,key=None\) | 产出由两个元素组成的元素，形式为 \(key,group\)，其中 key 是分组标准，group 是生成器，用于产出分组里的元素 |
| 内置        | reversed\(seq\)        | 从后向前，倒序产出 seq 中的元素；seq 必须是序列，或者是实现了 \_\_reversed\_\_ 特殊方法的对象      |
| itertools | tee\(it, n=2\)         | 产出一个由 n 个生成器组成的元组，每个生成器用于单独产出输入的可迭代对象中的元素                         |


###### 示例 14-21　itertools.groupby 函数的用法

In [0]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x7fc02c1065f8>),
 ('A', <itertools._grouper at 0x7fc02c106cf8>),
 ('G', <itertools._grouper at 0x7fc02c106e48>)]

In [0]:
for char, group in itertools.groupby('LLLLAAAGG'):
  print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [0]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear',
       'bat', 'dolphin', 'shark', 'lion']
animals

['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']

In [0]:
animals.sort(key=len)  # 为了使用 groupby 函数，必须要排序输入
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [0]:
for length, group in itertools.groupby(animals, len):
  print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [0]:
for length, group in itertools.groupby(reversed(animals), len):
  print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


###### 示例 14-22　itertools.tee 函数产出多个生成器，每个生成器都可以产出输入的各个元素

In [0]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x7fc02c179488>, <itertools._tee at 0x7fc02c10aa08>]

In [0]:
g1, g2 = itertools.tee('ABC')
next(g1)

'A'

In [0]:
next(g2)

'A'

In [0]:
next(g2)

'B'

In [0]:
list(g1)

['B', 'C']

In [0]:
list(g2)

['C']

In [0]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 14.10　Python 3.3中新出现的句法：yield from

- `yield from` 可以代替内层的 `for` 循环

In [0]:
def chain(*iterables):
  for it in iterables:
    for i in it:
      yield i

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [0]:
def chain(*iterables):
  for it in iterables:
    yield from it

In [0]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

## 14.11 可迭代的归约函数

###### 表14-6：读取迭代器，返回单个值的内置函数

| 模块        | 函数                                | 说明                                                                       |
|-----------|-----------------------------------|--------------------------------------------------------------------------|
| 内置        | all\(it\)                         | it 中的所有元素都为真值时返回 True，否则返回 False；all\(\[\]\) 返回 True                     |
| 内置        | any\(it\)                         | 只要 it 中有元素为真值就返回 True，否则返回 False；any\(\[\]\) 返回 False                    |
| 内置        | max\(it, \[key=,\] \[default=\]\) | 返回 it 中值最大的元素；\*key 是排序函数，与 sorted 函数中的一样；如果可迭代的对象为空，返回 default          |
| 内置        | min\(it,\[key=,\]\[default=\]\)   | 返回 it 中值最小的元素；\#key 是排序函数，与 sorted 函数中的一样；如果可迭代的对象为空，返回 default          |
| functools | reduce\(func,it,\[initial\]\)     | 把前两个元素传给 func，然后把计算结果和第三个元素传给 func，以此类推，返回最后的结果；如果提供了initial，把它当作第一个元素传入 |
| （内置）      | sum\(it,start=0\)                 | it 中所有元素的总和，如果提供可选的 start，会把它加上（计算浮点数的加法时，可以使用 math\.fsum 函数提高精度）        |

- \* 也可以像这样调用：max(arg1, arg2, ..., [key=?])，此时返回参数中的最大值  
- \# 也可以像这样调用：min(arg1, arg2, ..., [key=?])，此时返回参数中的最小值

###### 示例 14-23　把几个序列传给 `all` 和 `any` 函数后得到的结果

In [0]:
all([1, 2, 3])

True

In [0]:
all([1, 0, 3])

False

In [0]:
all([])

True

In [0]:
any([1, 2, 3])

True

In [0]:
any([1, 0, 3])

True

In [0]:
any([0, 0, 0])

False

In [0]:
any([])

False

In [0]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

In [0]:
next(g)  # any(g) 遇到 7 即返回 True

8

## 14.12 深入分析 `iter` 函数

- `iter` 函数的另一个用法为
  - 传入两个参数，使用常规的函数或任何可调用的对象创建迭代器
  - 在使用时，第一个参数必须是可调用的对象，用于不断调用（没有参数），产出各个值；
  - 第二个值是哨符，这是个标记值，当可调用的对象返回这个值时，触发迭代器抛出 `StopIteration` 异常，而不产出哨符

- 示例： 用 `iter` 掷骰子，直至掷出 1 点为止

In [1]:
from random import randint


def d6():
  return randint(1, 6)


d6_iter = iter(d6, 1)
print(d6_iter)

for roll in d6_iter:
  print(roll)

6
5
5
3


- 逐行读取文件，直至遇到空行或到达文件末尾
  - ```python
    with open('mydata.txt') as fp:
      for line in iter(fp.readline, '\n'):
        process_line(line)
  ```

## 14.13　案例分析：在数据库转换工具中使用生成器

- 使用生成器处理数据库时，可以把记录看成数据流，这样消耗的内存量最低，而且不管数据有多大都能处理

## 14.14　把生成器当成协程

- 生成器用于生成供迭代的数据
- 协程是数据的消费者
- 为了避免脑袋炸裂，不能把这两个概念混为一谈
- 协程与迭代无关
- 虽然在协程中会使用 `yield` 产出值，但这与迭代无关